In [4]:
from typing import Iterator
from agno.agent import Agent, RunResponse
from agno.models.openai.like import OpenAILike
from agno.utils.pprint import pprint_run_response
from agno.vectordb.lancedb import LanceDb
from agno.vectordb.search import SearchType
from agno.embedder.openai import OpenAIEmbedder
from pathlib import Path

import os
from dotenv import load_dotenv
load_dotenv()
api_key=os.getenv("QWEN_API_KEY")
base_url=os.getenv("QWEN_API_BASE_URL")
local_base_url = 'http://192.168.0.166:8000/v1'
local_model_name = 'Qwen3-235B'
model_name = 'qwen-plus-latest'
embedding_model_id = 'text-embedding-v3'

local_settings = {
  'api_key' : '123',
  'base_url' : local_base_url,
  'id' : local_model_name
}

qwen_settings = {
  'api_key' : api_key,
  'base_url' : base_url,
  'id' : model_name
}

settings = qwen_settings

## 1. 设置向量数据库

In [7]:
vector_db = LanceDb(
    table_name="base_vectors",
    uri="tmp/lancedb",
    search_type=SearchType.hybrid,
    embedder=OpenAIEmbedder(id=embedding_model_id,api_key=api_key,base_url=base_url, dimensions=1024),
)

### 1.1 填充向量数据库（根据不同的文件类型，调用对应的向量数据库，然后作为上下文检索提供给LLM）

In [22]:
from agno.knowledge.website import WebsiteKnowledgeBase
knowledge_base = WebsiteKnowledgeBase(
    urls=["https://docs.agno.com/introduction"],
    # Number of links to follow from the seed URLs
    max_links=10,
    # Table name: ai.website_documents
    vector_db=vector_db,
)

第一次运行的时候相当于从源中读取数据加载到向量数据库中，所以只需要执行一次，数据库中就存储了相关数据

In [23]:
knowledge_base.load(recreate=False)

INFO Loading knowledge base

## 2. 加载数据库

使用基类`AgentKnowledge`即可完成任一数据库的加载

In [14]:
from agno.knowledge import AgentKnowledge
knowledge = AgentKnowledge(vector_db=vector_db)

In [15]:
agent = Agent(model=OpenAILike(**settings),knowledge=knowledge,telemetry=False)

In [18]:
response = agent.run(message='what is agno?')

INFO Found 5 documents

In [19]:
print(response.content)

Agno is a full-stack framework for building Multi-Agent Systems with memory, knowledge, and reasoning capabilities. It is designed to help engineers and researchers develop agentic systems across five levels of complexity:

1. **Level 1**: Agents with tools and instructions.
2. **Level 2**: Agents with knowledge and storage.
3. **Level 3**: Agents with memory and reasoning.
4. **Level 4**: Agent teams that can reason and collaborate.
5. **Level 5**: Agentic workflows with state and determinism.

### Key Features of Agno
- **Model Agnostic**: Supports over 23+ model providers without lock-in.
- **Highly Performant**: Fast agent instantiation (~3μs) and low memory usage (~6.5KiB).
- **Reasoning Support**: Includes multiple approaches like reasoning models, tools, or chain-of-thought methods.
- **Natively Multi-Modal**: Accepts text, image, audio, and video as input and output.
- **Multi-Agent Architecture**: Industry-leading support for multi-agent collaboration (Agent Teams).
- **Built-

In [27]:
for document in knowledge_base.document_lists:
    print(document)

WARNING  HTTP status error while crawling https://docs.agno.com/: Redirect response '308 Permanent Redirect' for   
         url 'https://docs.agno.com/'                                                                              
         Redirect location: '/introduction'                                                                        
         For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/308

[Document(content='Agno home page Search... Ask AI Discord Community agno-agi / agno agno-agi / agno Search... Navigation Introduction What is Agno? User Guide Examples Workspaces FAQs API reference Changelog Introduction What is Agno? Your first Agents Multi Agent Systems Agent Playground Monitoring & Debugging Community & Support Concepts Agents Teams Models Tools Reasoning Memory Knowledge Chunking Vector DBs Storage Embeddings Evals Workflows Applications Other Agent UI Agent API Observability How to Install & Setup Contributing to Agno Migrate from Phidata to Agno Introduction What is Agno? Copy page Agno is a full-stack framework for building Multi-Agent Systems with memory, knowledge and reasoning. Engineers and researchers use Agno to build the 5 levels of Agentic Systems: Level 1: Agents with tools and instructions ( example ). Level 2: Agents with knowledge and storage ( example ). Level 3: Agents with memory and reasoning ( example ). Level 4: Agent Teams that can reason and